## Applying viterbi Algorithm : to consider tag sequence information : DP approach
* in this notebook we ll apply viterbi algorithm on test data , and conclude with accuracy on test data

In [ ]:
# importing necessary pacakges
import pandas as pd
import numpy as np
import pickle

### loading  test featurized data matrix to work on

In [ ]:
df = pd.read_csv('./final_df_test.csv')
df.head()

### adding some more feautres

In [ ]:
length = []
for s in df['word']:
    length.append(len(s))

length_2 = np.asarray(length).reshape(-1,1)
df['length'] = length_2

In [ ]:
hyphen = []
all_cap = []
sp_char = []
cap_dot = []
number = []
st_alpha_dot = []
e_day = []
for i in df['word']:
    if('-' in i):
        hyphen.append(1)
    else:
        hyphen.append(0)
    if(i.isupper()):
        all_cap.append(1)
    else:
        all_cap.append(0)
    if(('$' in i or ',' in i or '%' in i or '!' in i or '\'' in i or '"' in i) \
       and len(i) == 1):
        sp_char.append(1)
    else:
        sp_char.append(0)
    if(i[0].isupper() and i.endswith('.')):
        cap_dot.append(1)
    else:
        cap_dot.append(0)
    if(i[0].isdigit() and i[-1].isdigit() and ('.' in i or ',' in i)):
        number.append(1)
    else:
        number.append(0)
    if(i[0].isalpha() and '.' in i):
        st_alpha_dot.append(1)
    else:
        st_alpha_dot.append(0)
    if(i.endswith('day')):
        e_day.append(1)
    else:
        e_day.append(0)
#print('\'')

In [ ]:
df['hyphen'] = hyphen
df['all_cap'] = all_cap
df['sp_char'] = sp_char
df['cap_dot'] = cap_dot
df['number'] = number
df['st_alpha_dot'] = st_alpha_dot
df['e_day'] = e_day

In [ ]:
tags = df.tag.unique()
len(tags)

### creating a datamatrix 'X' and target label coln vector 'y'

In [ ]:
data = np.array(df)

x = data[:,2:]
x.shape
y = df['tag']
y.shape

### adding prev and next context as features

In [ ]:
no_cols = x.shape[1] * 5
features = np.zeros((47380,no_cols))

b = no_cols / 5
b = int(b)

In [ ]:
# taking prev and next words as feature vectors : appending the stored feature vectors of the same

for i in range(x.shape[0]):
    if(i==0):
        features[i][b * 2: b* 3] = x[i]
        features[i][b* 3:b* 4] = x[i+1]
        features[i][b* 4:b* 5] = x[i+2]
    
    elif(i == 1):
        features[i][b* 1:b* 2] = x[i-1]
        features[i][b* 3:b* 4] = x[i+1]
        features[i][b* 2:b* 3] = x[i]
        features[i][b* 4:b* 5] = x[i+2]
        
    elif(i == x.shape[0]-1):
        features[i][0:b* 1] = x[i-2]
        features[i][b* 1:b* 2] = x[i-1]
        features[i][b* 2:b* 3] = x[i]

    elif(i == x.shape[0]-2):
        features[i][b* 3:b* 4] = x[i+1]
        features[i][0:b* 1] = x[i-2]
        features[i][b* 1:b* 2] = x[i-1]
        features[i][b* 2:b* 3] = x[i]
    
    else:
        features[i][0:b* 1] = x[i-2]
        features[i][b* 1:b* 2] = x[i-1]
        features[i][b* 2:b* 3] = x[i]
        features[i][b* 3:b* 4] = x[i+1]
        features[i][b* 4:b* 5] = x[i+2]

In [ ]:
feature_vect = features[:-3]
feature_vect.shape

### label encoding

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(y)
Y = le.transform(y)

### Loading trained softmax model for both c and c^2 no of classes , and also loading mapping of class label to numerical encoding

In [ ]:
uni_pairs = pickle.load(open('./pair_unigrams.pkl', 'rb'))
bi_pairs = pickle.load(open('./pair_bigrams.pkl', 'rb'))
soft_uni = pickle.load(open('./softmax.pkl', 'rb'))
soft_bi = pickle.load(open('./softmax_dual.pkl', 'rb'))

### creating list of sentences on which we ll apply viterbi algo : dynamic programming approach

In [ ]:
# creating list of sentences 
list_sent = []
l = []
for word in df.word:
    l.append(word)
    if(word == '.'):
        list_sent.append(l)
        l = []
    

In [ ]:
sent = list(df['word'][1712:1742])

In [ ]:
feature_vect[45:45]

In [ ]:
uni_pairs
res = dict((v,k) for k,v in uni_pairs.items())

## Viterbi Alogrithm Implementation 

In [ ]:
feat_vect_cnt = 0

total_pred_cnt = 0
total_actual_cnt = 0

for sent in list_sent:
    
    table = np.zeros((44,len(sent)))
    tindex = np.zeros((44,len(sent)), dtype=int)

    # taking standalone probs of first word of a sent
    x = soft_uni.predict_proba(feature_vect[feat_vect_cnt].reshape(1,-1))

    #intialization : storing standalone probs of w1 in column 1
    for i in range(x.shape[1]):
        table[i][0] = x[0][i]

    tindex[:,0:1] = -1
    #print(table[:,0:1].shape)
    
    # viterbi algorithm : dp logic
    word = 1    # starting with 2nd coln : word2
    while(word < len(sent)):
        for tag in range(0,44):
            t1name = res[tag]
            prob2 = soft_bi.predict_proba(feature_vect[word + feat_vect_cnt].reshape(1,-1))
            mulmax = 0
            index = -1
            #print(prob2[0])
            for t2 in range(0,44):     # loop for first tag
                tag_pair = " ".join([res[t2],t1name])
                    #print(tag_pair)
                if(tag_pair in bi_pairs):           # if pair is thr then use prob else it ll store prev calcultd mulmax and index
                    sec_term = prob2[0][bi_pairs[tag_pair]]
                    #print(" a " ,sec_term)
                    f_term = table[t2][word-1]
                    #print(f_term)                 # pr(ti/w1)
                    mul_term = f_term * sec_term
                    if(mul_term > mulmax):
                        mulmax = mul_term
                        index = t2
            table[tag][word] = mulmax        # storing max prob out of 44 pssble probs
            tindex[tag][word] = index       # storing index in tindex

        word += 1
        
    
    
    print('start of sent : ',total_actual_cnt)
    
    
    # storing tag sequences tht maximizes the probability in reverse order 
    j = tindex.shape[1] - 1
    #np.max(table[:, -1:])
    k = np.argmax(table[:, -1:])
    tag_index = []
    tag_index.append(k)
    while j > 0:
        k = tindex[k][j]
        j -= 1
        tag_index.append(k)
        
    # actual ordering
    actual = []
    for i in reversed(tag_index):
        actual.append(res[i])
    total_actual_cnt += len(actual)
    
    
    # calculating accuracy
    c = 0
    j = feat_vect_cnt
    for i in range(len(actual)):
        if df.tag[j] == actual[i]:
            c += 1
        j += 1
    total_pred_cnt += c

    # increasing feature_vector count to starting of the next sent
    feat_vect_cnt += len(sent)
    
    #print(c/len(actual))
    
accuracy = (total_pred_cnt/total_actual_cnt)*100
print("accuracy : " , accuracy)

#### Accuracy :  67.4441184541022